In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## Dataframe processing with cuDF

<!--
01234567890123456789012345678901234567890123456789012345678901234567890123456789
-->

[cuDF](https://github.com/rapidsai/cudf) provides a drop in replacement for the 
[pandas](https://pandas.pydata.org/) data analysis package.

cuDF makes the compute resourcesin GPGPUs accessible when manipulating
time-series and matrix data. It is build on and compatible with cupy and numpy 
and supports both numerical and textual data.

This section assumes that you are already familar with pandas and aims to
demonstrate how cuDF can be used instead.

A very nice, extended presentation similar to this was presented at
[NERSC](https://www.nersc.gov/users/training/events/rapids-hackathon/) and parts
of this material are based on that prior presentation.

### cuDF as a drop in replacement for Pandas

<!--
01234567890123456789012345678901234567890123456789012345678901234567890123456789
-->

cuDF implements many of pandas' interfaces and in many cases it can be used as a
drop-in replacement for Pandas by simply changing from `import pandas` to
`import cudf`, but see its [compatibility notes](https://docs.rapids.ai/api/cudf/stable/basics/PandasCompat.html).

First let's create a simple dataframe with two columns named "key" and "value"

In [204]:
import pandas as pd

df = pd.DataFrame()
df['value1'] = [0, 0, 2, 2, 3]
df['value2'] = [float(i + 10) for i in range(5)]
print(df)

   value1  value2
0       0    10.0
1       0    11.0
2       2    12.0
3       2    13.0
4       3    14.0


and compute some reduction over a column of data

In [205]:
df['value2'].sum()

60.0

Next the same code using cuDF

In [206]:
import cudf as cudf

df = cudf.DataFrame()
df['value1'] = [0, 0, 2, 2, 3]
df['value2'] = [float(i + 10) for i in range(5)]
print(df)

   value1  value2
0       0    10.0
1       0    11.0
2       2    12.0
3       2    13.0
4       3    14.0


In [207]:
df['value2'].sum()

60.0

However cuDF data frames are stored in GPU memory and use `cupy` under the hood:

In [208]:
print("cuDF")
print(type(df))
print(type(df['value2'].values))

cuDF
<class 'cudf.core.dataframe.DataFrame'>
<class 'cupy.core.core.ndarray'>


### Conversion to / from pandas, cupy or numpy arrays

If needed cuDF objects can be converted and from `pandas` and `numpy`.

In [209]:
pandas_df = df.to_pandas()
print("Pandas from cuDF")
print(type(pandas_df))
print(pandas_df)

Pandas from cuDF
<class 'pandas.core.frame.DataFrame'>
   value1  value2
0       0    10.0
1       0    11.0
2       2    12.0
3       2    13.0
4       3    14.0


In [210]:
df = cudf.from_pandas(pandas_df)
print("cuDF from Pandas")
print(type(df))
print(df)

cuDF from Pandas
<class 'cudf.core.dataframe.DataFrame'>
   value1  value2
0       0    10.0
1       0    11.0
2       2    12.0
3       2    13.0
4       3    14.0


In [211]:
cupy_ndarray =  df.values
print("cupy from cudf")
print(type(cupy_ndarray))
print(cupy_ndarray)

print("numpy from cudf")
numpy_ndarray = df.values.get()
print(type(numpy_ndarray))
print(numpy_ndarray)

cupy from cudf
<class 'cupy.core.core.ndarray'>
[[ 0. 10.]
 [ 0. 11.]
 [ 2. 12.]
 [ 2. 13.]
 [ 3. 14.]]
numpy from cudf
<class 'numpy.ndarray'>
[[ 0. 10.]
 [ 0. 11.]
 [ 2. 12.]
 [ 2. 13.]
 [ 3. 14.]]


### Operating on cuDF data

<!--
01234567890123456789012345678901234567890123456789012345678901234567890123456789
-->
cuDF supports custom, user supplied operations on data that use `numba` jit
compiler to translate Python code to GPU code.

In [113]:
from numba import cuda
import numpy as np
import math

In [212]:
np.random.seed(42)

df = cudf.DataFrame()
data_len = 1000
df['x'] = np.random.normal(10., 1., data_len)
df['y'] = np.random.normal(10., 1., data_len)
df['z'] = np.random.normal(10., 1., data_len)

df.head(10)

,x,y,z
0,10.496714,11.399355,9.324822
1,9.861736,10.924634,9.855481
2,10.647689,10.059630,9.207580
3,11.523030,9.353063,9.692038
4,9.765847,10.698223,8.106385
5,9.765863,10.393485,10.213294
6,11.579213,10.895193,10.001205
7,10.767435,10.635172,9.182911
8,9.530526,11.049553,10.659246
9,10.542560,9.464765,10.937570


### Point wise operations using cuDF

In [12]:
def my_pow2(x):
    return x**2

out = df['x'].applymap(my_pow2)[:10]

print(out[:10])

0    110.181008
1     97.253831
2    113.373271
3    132.780217
4     95.371760
5     95.372081
6    134.078169
7    115.937651
8     90.830918
9    111.145572
Name: x, dtype: float64


### Point wise operations using numba directly

In [216]:
@cuda.jit
def my_pow2(x, out):
    i = cuda.grid(1)
    if i < x.size: # boundary guard
        out[i] = x[i]**2

out    = cudf.DataFrame()
out['out'] = np.zeros(len(df['x']), dtype=np.float64)
my_pow2.forall(len(df['x']))(df['x'], out['out'])

print(out[:10])

          out
0  110.181008
1   97.253831
2  113.373271
3  132.780217
4   95.371760
5   95.372081
6  134.078169
7  115.937651
8   90.830918
9  111.145572


### Row wise operations using cuDF

<!--
01234567890123456789012345678901234567890123456789012345678901234567890123456789
-->
A simpleway to process data is to apply a given function row-wise to data and
append the result to the dataframe.

In [14]:
def my_sqrt(in1, in2, in3, out, thread, n):
    for i, (x,y,z) in enumerate(zip(in1, in2, in3)):
        out[i] = (x**n+y**n+z**n)**(1./n)
        thread[i] = cuda.threadIdx.x
    
out = df.apply_rows(my_sqrt,
                    incols={'x':'in1', 'y':'in2', 'z':'in3'},
                    outcols=dict(out=np.float64, thread=np.int32),
                    kwargs=dict(n=2))
print(out[:10])

           x          y          z        out  thread
0  10.496714  11.399355   9.324822  18.085315       0
1   9.861736  10.924634   9.855481  17.712480       1
2  10.647689  10.059630   9.207580  17.301704       2
3  11.523030   9.353063   9.692038  17.725564       3
4   9.765847  10.698223   8.106385  16.599314       4
5   9.765863  10.393485  10.213294  17.541607       5
6  11.579213  10.895193  10.001205  18.783171       6
7  10.767435  10.635172   9.182911  17.702271       7
8   9.530526  11.049553  10.659246  18.070502       8
9  10.542560   9.464765  10.937570  17.898541       9


<!--
01234567890123456789012345678901234567890123456789012345678901234567890123456789
-->
Depending on how the number of threads available on the GPU and the size of the
array, a thread may end up processing multiple rows, that themselves may or may
not be coongiuous.

In [15]:
print(out[out['thread'] == 0])
print(out[out['thread'] == 511])

             x          y          z        out  thread
0    10.496714  11.399355   9.324822  18.085315       0
512   9.761052   9.283178  10.548884  17.109485       0
            x         y          z        out  thread
511  9.949762  9.806341  10.547265  17.504481     511


### Row wise operations using numba directly

In [213]:
@cuda.jit
def my_sqrt(x, y, z, out, thread, n):
    i = cuda.grid(1)
    if i < x.size: # boundary guard
        out[i] = (x[i]**n+y[i]**n+z[i]**n)**(1./n)
        thread[i] = cuda.threadIdx.x

out    = cudf.DataFrame()
out['out'] = np.zeros(len(df['x']), dtype=np.float64)
out['thread'] = np.zeros(len(df['x']), dtype=np.int32)

my_sqrt.forall(len(df['x']))(df['x'], df['y'], df['z'], out['out'], out['thread'], 2.)

print(out[:10])

         out  thread
0  18.085315       0
1  17.712480       1
2  17.301704       2
3  17.725564       3
4  16.599314       4
5  17.541607       5
6  18.783171       6
7  17.702271       7
8  18.070502       8
9  17.898541       9


In [17]:
print(out[out['thread'] == 0])
print(out[out['thread'] == 511])

           out  thread
0    18.085315       0
640  18.257649       0
           out  thread
511  17.504481     511


### More complex manipulations

In [183]:
df = cudf.DataFrame()
data_len = 1000
df['x'] = np.arange(1., data_len+1)

df.head(10)

,x
0,1.0
1,2.0
2,3.0
3,4.0
4,5.0
5,6.0
6,7.0
7,8.0
8,9.0
9,10.0


#### Moving average using cudf

In [184]:
def my_moving_avg(window):
    total = 0.
    for a in window:
        total += a**2
    total /= len(window)
    return total
    
dfr = df['x'].rolling(window=3,  center=True)
out = dfr.apply(my_moving_avg)
    
print(out[:10])

0           <NA>
1    4.666666667
2    9.666666667
3    16.66666667
4    25.66666667
5    36.66666667
6    49.66666667
7    64.66666667
8    81.66666667
9    100.6666667
Name: x, dtype: float64


#### Moving average using numba directly

In [185]:
@cuda.jit
def my_moving_avg(x, out):
    i = cuda.grid(1)
    if i >= 1 and i < x.size-1: # boundary guard
        total = 0.
        for j in range(i-1, i+2):
            total += x[j]**2
        total /= 3
    else:
        total = math.nan
    out[i] = total

out    = cudf.DataFrame()
out['out'] = np.zeros(len(df['x']), dtype=np.float64)

my_moving_avg.forall(len(df['x']))(df['x'], out['out'])

print(out[:10])

          out
0         NaN
1    4.666667
2    9.666667
3   16.666667
4   25.666667
5   36.666667
6   49.666667
7   64.666667
8   81.666667
9  100.666667


## Manipulating text with cuDF

cudf has support to handle strings in dataframes, patterend after Pandas. For details please consult the [documentation](https://docs.rapids.ai/api/cudf/stable/api.html#strings).

In [115]:
df = cudf.DataFrame()
df['string'] = ['Mary', 'had', 'a', 'little', 'lamb']
df['data'] = [68.534, 35.5, 4., 9. , -5174.42050]
df[df['string'].str.contains('^[a-z]*$')]

,string,data
1,had,35.5000
2,a,4.0000
3,little,9.0000
4,lamb,-5174.4205


#### Counting unique words

In [117]:
# get text of Hamlet
import requests
url = 'https://www.gutenberg.org/ebooks/1787.txt.utf-8'
content = requests.get(url).content.decode('utf-8')

In [118]:
# strip out project Gutenberg header and footer
lines = content.split('\r\n')
# strip out license etc...
for first,line in enumerate(lines):
    if line == "ACT I. Scene I.":
        break
for last,line in enumerate(lines):
    if line == "THE END":
        break
lines = lines[first:last+1]

#### Count words using Python and the CPU

In [221]:
import re
def get_word_count(text):
    words = {}
    for line in text:
        for w in re.sub("[^a-zA-Z0-9]", " ", line).split():
            try:
                words[w] += 1
            except KeyError:
                words[w] = 1
    return sorted(words.items(), key=lambda w: w[1], reverse=True)

%time get_word_count(lines)[:10]

CPU times: user 59.6 ms, sys: 823 µs, total: 60.5 ms
Wall time: 58.5 ms


[('the', 928),
 ('and', 694),
 ('to', 631),
 ('of', 626),
 ('I', 608),
 ('you', 486),
 ('a', 477),
 ('my', 444),
 ('in', 412),
 ('it', 359)]

#### Count words using cuDF and the GPU

In [222]:
# stolen from https://gist.github.com/VibhuJawa/df3583ed553ac84b990619d7c49f2a73
# which is used in https://medium.com/rapids-ai/show-me-the-word-count-3146e1173801
def get_word_count(text):
    """
        returns the count of input strings
    """ 
    ## Tokenize: convert sentences into a long list of words
    ## Get counts: Groupby each token to get value counts

    df = cudf.DataFrame()
    # tokenize sentences  into a nvstrings instance using nvtext.tokenize()
    # converting it into a single tall data-frame
    df['string'] = text.str.filter_alphanum(' ').str.tokenize()
    # Using Group by to do a value count for string columns

    df['counts'] = np.dtype('int32').type(0)
    
    res = df.groupby('string').count()
    res = res.reset_index(drop=False).sort_values(by='counts', ascending=False)
    return res

text = cudf.Series(lines)
%time get_word_count(text)[:10]

CPU times: user 28.2 ms, sys: 0 ns, total: 28.2 ms
Wall time: 28 ms


,string,counts
1040,the,928
4789,and,694
4364,to,631
1616,of,626
4599,I,608
152,you,486
1348,a,477
265,my,444
3224,in,412
4385,it,359


#### Count words using C++ and the CPU

In [201]:
%%writefile wc.cc
#include <algorithm>
#include <algorithm>
#include <cctype>
#include <iostream>
#include <iterator>
#include <list>
#include <unordered_map>
#include <sstream>
#include <string>
#include <vector>

#include <sys/time.h>

// bunch of typedefs to make types readable
typedef std::unordered_map<std::string, int> count_t;         // raw word count
typedef std::list<std::string> lines_t;                       // list of lines from file
typedef std::vector<std::pair<std::string, int> > countvec_t; // sortable container for words

// helper function to compare word counts
bool cmp(const std::pair<std::string, int>&a,
         const std::pair<std::string, int>&b) {
  return a.second > b.second;
}

countvec_t get_word_count(const lines_t& lines) {
  count_t count;

  // remove non-alnum chars, split each line into words, and count those words
  for(auto line: lines) {
    for(auto &c: line) {
      if(!isalnum(c))
        c = ' ';
    }

    std::stringstream iss(line);
    std::string word;
    while(iss >> word) {
      count[word] += 1;
    }
  }

  // now sort by number of occurrences
  countvec_t countvec(count.size());
  std::move(count.begin(), count.end(), countvec.begin());
  std::sort(countvec.begin(), countvec.end(), cmp);

  return countvec;
}

int main(void) {
  // read in all lines from file
  lines_t lines;
  while(!std::cin.eof()) {
    std::string line;
    std::getline(std::cin, line);
    lines.push_back(line);
  }

  // time actual word count and list construction
  struct timeval start, end;
  gettimeofday(&start, NULL);
  countvec_t countvec = get_word_count(lines);
  gettimeofday(&end, NULL);

  // all done, show results
  double dt = (end.tv_sec - (double)start.tv_sec) + (end.tv_usec - (double)start.tv_usec)/1e6;
  std::cout << "took " << dt*1e3 << "ms\n";

  for(auto c: countvec) {
    std::cout << c.first << ": " << c.second << "\n";
  }
    
  return 0;
}

Overwriting wc.cc


In [202]:
! g++ -O3 -std=c++11 -o wc wc.cc
import os
with os.popen("./wc | head -n 10", "w") as wc:
    wc.write("\n".join(lines))

took 21.818ms
the: 928
and: 694
to: 631
of: 626
I: 608
you: 486
a: 477
my: 444
in: 412
